###Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
users = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'movieId': 'count'})
users

timestamp             movieId
               min         max   count
userId                                
1        964980499   965719662     232
2       1445714835  1445715340      29
3       1306463323  1306464293      39
4        945078428  1007574542     216
5        847434747   847435337      44
...            ...         ...     ...
606     1171228846  1368460577    1115
607      963078392   997847387     187
608     1117161730  1189563917     831
609      847220802   847221080      37
610     1479541963  1495959411    1302

[610 rows x 3 columns]

In [4]:
users[users['movieId']['count'] >= 100]

timestamp             movieId
               min         max   count
userId                                
1        964980499   965719662     232
4        945078428  1007574542     216
6        845553109   845556915     314
7       1106635416  1176181731     152
10      1455301553  1455619275     140
...            ...         ...     ...
605     1277094764  1277177012     221
606     1171228846  1368460577    1115
607      963078392   997847387     187
608     1117161730  1189563917     831
610     1479541963  1495959411    1302

[248 rows x 3 columns]

In [5]:
users['Lifetime'] = users.apply(lambda row: row['timestamp']['max'] - row['timestamp']['min'], axis=1)

In [6]:
avg_life_time = round(users['Lifetime'].mean())
avg_life_time

20243099

In [7]:
timedelta(seconds = avg_life_time).days

234

###Задание 2

####1. Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

####2. Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [8]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [9]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [10]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [11]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [12]:
rzd.merge(auto, how = 'outer', on = 'client_id').merge(air, how = 'outer', on = 'client_id').fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [13]:
(client_base.merge(rzd, how = 'outer', on = 'client_id').
     merge(auto, how = 'outer', on = 'client_id').
         merge(air, how = 'outer', on = 'client_id').fillna(0))

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


###Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

По двум координатам (долготе и широте) можно опредилить местоположения пользователя во рвемя визитов и факта покупки. Пользователи могу совершать один и тот же путь каждый день. Благодаря этим данным  можно разместить необходимую рекламму по пути следования пользователя.